In [ ]:
import open3d as o3d
import sys
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import imageio
from tqdm import tqdm

In [ ]:
path = "../data/KITTI/depth/depth_selection/val_selection_cropped/"

#### RGB-D image

In [ ]:
ind = 5
seq = "2011_09_26_drive_0002_sync"
# ind = np.random.randint(0, 553)

color_raw = o3d.io.read_image(os.path.join(path, "image/%s_image_%010d_image_02.png" % (seq, ind)))
depth_raw = o3d.io.read_image(os.path.join(path, "groundtruth_depth/%s_groundtruth_depth_%010d_image_02.png" % (seq, ind)))
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('grayscale image')
plt.imshow(rgbd_image.color, cmap="gray")
plt.subplot(1, 2, 2)
plt.title('depth image')
plt.imshow(rgbd_image.depth, cmap="gray")
plt.show()

#### Intrinsics (camera matrix)

In [ ]:
w, h = np.asarray(rgbd_image.color).shape

K = np.loadtxt(os.path.join(path, "intrinsics/%s_image_%010d_image_02.txt" % (seq, ind))).reshape(3, 3)
intrinsics = o3d.camera.PinholeCameraIntrinsic(width=w, height=h,
                                               fx=K[0, 0], fy=K[1, 1], cx=K[0, 2], cy=K[1, 2])

In [ ]:
# Usually has shape:
# array([[fx,   0., cx],
#       [  0. , fy, cy],
#       [  0. , 0., 1.]]
# Reference:
# (1) TDV slides: http://cmp.felk.cvut.cz/cmp/courses/TDV/2021W/lectures/tdv-2021-02.pdf
# (2) TDV lecture video (around 38:30): http://cmp.felk.cvut.cz/cmp/courses/TDV/2021W/lectures/tdv-2021-02.mp4

K

#### Point cloud

In [ ]:
# pts = np.load(path+f"point_clouds/{ind}.npy")

# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(pts)
# o3d.visualization.draw_geometries([pcd])

#### Create point cloud from RGBD image

In [ ]:
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsics)

# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

o3d.visualization.draw_geometries([pcd])
print(pcd)

In [ ]:
np.asarray(pcd.points)

#### Dataset

In [ ]:
from supervised_depth_correction.data import Dataset
from tqdm import tqdm

device = torch.device('cpu')

subseq = "2011_09_26_drive_0002_sync"
depth_set = "val"

ds = Dataset(subseq, depth_type="dense", depth_set="val", camera='left', zero_origin=False, device=device)
ds_pred = Dataset(subseq, depth_type="pred", depth_set="val", camera='left', zero_origin=False, device=device)
assert len(ds) > 0
assert len(ds) == len(ds_pred)

In [ ]:
i = np.random.randint(0, len(ds))
colors, depths, K, poses = ds[i]
_, depths_pred, _, _ = ds_pred[i]

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.title('grayscale image')
plt.imshow(colors.squeeze() / 255., cmap="gray")
plt.subplot(1, 3, 2)
plt.title('depth image')
plt.imshow(depths.squeeze(), cmap="gray")
plt.subplot(1, 3, 3)
plt.title('predicted depth image')
plt.imshow(depths_pred.squeeze(), cmap="gray")
plt.show()